In [9]:
#PREVIOUS TEAM
import asyncio
from autogen_agentchat.agents import AssistantAgent
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_agentchat.messages import TextMessage
from dotenv import load_dotenv
load_dotenv()
import os

key = os.getenv("Groq_api_key")

model_client =  OpenAIChatCompletionClient(
    base_url="https://api.groq.com/openai/v1",
    model="llama-3.3-70b-versatile",
    api_key = key,
    model_info={
        "family":'llama',
        "vision" :False,
        "function_calling":True,
        "json_output": True
    }
)

plot_agent = AssistantAgent(
    name="plot_agent",
    model_client=model_client,
    system_message="you are a creative write. generate a story plot on a small dog and a small kid"
)

character_agent = AssistantAgent(
    name="character_agent",
    model_client=model_client,
    system_message="develope character of the story. describe dog and kid in detail with their background"
)

ending_agent = AssistantAgent(
    name="ending_agent",
    model_client=model_client,
    system_message="write engaging endings and end the story with a twist"
)

from autogen_agentchat.teams import RoundRobinGroupChat

team = RoundRobinGroupChat(
    [plot_agent, character_agent, ending_agent], # list of agents 
    max_turns=3
     # max_turns: 3, # if its 1 then only plot agent will work, if 2 then plot and character agent will work, 3 then all 3 will work
    # termination_condition: condition to terminate the agents
)


async def test_team():
    task = TextMessage(
        content="write story about dog and boy",
        source='user'
    )

    result = await team.run(task=task)
    # since its a team we will have multiple messages so 
    for each_agent_message in result.messages:
        print(f"{each_agent_message}: {each_agent_message.content}")

await test_team()

c:\Users\shubu\Desktop\AutoGen Hardcode\venv\lib\site-packages\autogen_ext\models\openai\_openai_client.py:413: UserWarning: Missing required field 'structured_output' in ModelInfo. This field will be required in a future version of AutoGen.
  validate_model_info(self._model_info)


source='user' models_usage=None metadata={} content='write story about dog and boy' type='TextMessage': write story about dog and boy
source='plot_agent' models_usage=RequestUsage(prompt_tokens=59, completion_tokens=657) metadata={} content='**The Adventures of Max and Timmy**\n\nIn a small, sun-kissed town surrounded by green hills and friendly faces, there lived a little boy named Timmy. Timmy was a curious and adventurous six-year-old who loved exploring the outdoors. He had a best friend like no other, a small, fluffy dog named Max. Max was a playful and loyal terrier with a big heart, always by Timmy\'s side.\n\nOne sunny afternoon, as the warm breeze whispered through the trees, Timmy and Max decided to go on an adventure. They set out to explore the woods at the edge of their town, a place Timmy had always been fascinated by but never ventured into deeply. Max, sensing excitement, wagged his tail eagerly as he followed Timmy into the unknown.\n\nAs they wandered deeper into the 

### Observing Agent

In [ ]:
async for message in team.run_stream(task= "write a poem on genz kids in 10 words"):
    print(message)

source='user' models_usage=None metadata={} content='write a poem on genz kids' type='TextMessage'
source='plot_agent' models_usage=RequestUsage(prompt_tokens=2212, completion_tokens=387) metadata={} content="**The Generation of Wonder**\n\nIn a world of screens and endless space,\nA generation grows with a curious face,\nTheir eyes, a canvas of wonder and might,\nDrinking in the world, with a digital light.\n\nWith thumbs that dance and fingers that fly,\nThey navigate realms, both real and high,\nTheir minds, a maze of thoughts, so free and bright,\nA fusion of tech and innocence in sight.\n\nThey are the Gen Z kids, with hearts so bold,\nTheir spirits, a blend of old and new to unfold,\nThey crave connection, in a world of isolation,\nAnd seek community, with a digital sensation.\n\nWith TikTok tales and Instagram streams,\nThey share their stories, in a virtual dream,\nTheir voices, a chorus of laughter and cheer,\nA generation that's fearless, and always near.\n\nTheir passions, a

In [11]:
#checking type of messages
async for message in team.run_stream(task= "write a poem on genz kids in 10 words"):
    print(type(message))

<class 'autogen_agentchat.messages.TextMessage'>
<class 'autogen_agentchat.messages.TextMessage'>
<class 'autogen_agentchat.messages.TextMessage'>
<class 'autogen_agentchat.messages.TextMessage'>
<class 'autogen_agentchat.base._task.TaskResult'>


In [ ]:
# when we have to stop of handle the stop reason (catch hold of reason model has stopped)

from autogen_agentchat.base import TaskResult
async for message in team.run_stream(task= "write a poem on genz kids in 10 words"):
    if isinstance(message,TaskResult): # task result is type of result 
        print(f"Stop Reason: {message.stop_reason}") 
    else:
        print(message)


# when we are getting type of our message as a text message then will not have stop_reason

source='user' models_usage=None metadata={} content='write a poem on genz kids in 10 words' type='TextMessage'
source='plot_agent' models_usage=RequestUsage(prompt_tokens=4132, completion_tokens=12) metadata={} content='Gen Z kids sparkle with innovative minds and hearts freely.' type='TextMessage'
source='character_agent' models_usage=RequestUsage(prompt_tokens=4146, completion_tokens=12) metadata={} content='Gen Z kids shine with creativity and digital savvy spirits.' type='TextMessage'
source='ending_agent' models_usage=RequestUsage(prompt_tokens=4156, completion_tokens=12) metadata={} content='Gen Z kids shine with creativity and digital savvy spirits.' type='TextMessage'
Stop Reason: Maximum number of turns 3 reached.


In [18]:
# to see in a better manner
from autogen_agentchat.ui import Console

await Console(team.run_stream(task= "write a poem on genz kids in 10 words"))

---------- TextMessage (user) ----------
write a poem on genz kids in 10 words
---------- TextMessage (plot_agent) ----------
Gen Z kids shine with creativity and digital savvy spirits.
---------- TextMessage (character_agent) ----------
Gen Z kids shine bright with innovative digital young minds.
---------- TextMessage (ending_agent) ----------
Gen Z kids shine bright with innovative digital young minds.


TaskResult(messages=[TextMessage(source='user', models_usage=None, metadata={}, content='write a poem on genz kids in 10 words', type='TextMessage'), TextMessage(source='plot_agent', models_usage=RequestUsage(prompt_tokens=4196, completion_tokens=12), metadata={}, content='Gen Z kids shine with creativity and digital savvy spirits.', type='TextMessage'), TextMessage(source='character_agent', models_usage=RequestUsage(prompt_tokens=4210, completion_tokens=12), metadata={}, content='Gen Z kids shine bright with innovative digital young minds.', type='TextMessage'), TextMessage(source='ending_agent', models_usage=RequestUsage(prompt_tokens=4220, completion_tokens=12), metadata={}, content='Gen Z kids shine bright with innovative digital young minds.', type='TextMessage')], stop_reason='Maximum number of turns 3 reached.')